## 📖 Teoria: argc e argv

### Firma main()

```c
int main(int argc, char *argv[]) {
         └──┬──┘      └───┬────┘
        argument      argument
         count        vector
}
```

### Esempio

```bash
$ ./randip -n 100 -s 192.168.1.0/24
```

```c
argc = 5
argv[0] = "./randip"
argv[1] = "-n"
argv[2] = "100"
argv[3] = "-s"
argv[4] = "192.168.1.0/24"
argv[5] = NULL  ← Sempre NULL-terminated
```

## 📖 Teoria: getopt()

### Prototipo

```c
#include <unistd.h>

int getopt(int argc, char *argv[], const char *optstring);

// Variabili globali
extern char *optarg;  // Argomento opzione corrente
extern int optind;    // Indice prossimo argv da processare
```

### Option String Syntax

```
"n:s:oh"
 ││││└── -h (no argument)
 │││└─── -o (no argument)
 ││└──── -s: (required argument)
 │└───── -n: (required argument)
 └────── opzione senza ':' = nessun argomento
         opzione con ':' = argomento richiesto
         opzione con '::' = argomento opzionale
```

### Loop Pattern

```c
int opt;
while ((opt = getopt(argc, argv, "n:s:oh")) != -1) {
    switch (opt) {
        case 'n':  // optarg contiene "100"
            count = atoi(optarg);
            break;
        case 's':  // optarg contiene "192.168.1.0/24"
            subnet = optarg;
            break;
        case '?':  // Opzione sconosciuta
            print_usage();
            exit(1);
    }
}
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/08
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/08

# Copia tutti i file
cp ../07/*.h ../07/*.c .
rm -f test_*.c

# Aggiorna ip_types.h con ProgramOptions
cat >> ip_types.h << 'EOF'

/**
 * Opzioni programma da CLI
 */
typedef struct {
    size_t count;          // -n, --count: numero IP
    const char *subnet;    // -s, --subnet: subnet CIDR/range
    const char *output;    // -o, --output: file output
    bool only_public;      // -p, --public: solo IP pubblici
    bool show_classes;     // -c, --classes: mostra classi
    bool verbose;          // -v, --verbose: output dettagliato
    bool help;             // -h, --help: mostra help
} ProgramOptions;

/**
 * Valori default opzioni
 */
#define DEFAULT_COUNT 10
#define DEFAULT_OUTPUT "ips.txt"
EOF

echo "✅ ip_types.h aggiornato"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/08

# Crea cli.h
cat > cli.h << 'EOF'
#ifndef CLI_H
#define CLI_H

#include "ip_types.h"

/**
 * Inizializza opzioni con valori default
 */
void init_options(ProgramOptions *opts);

/**
 * Parse argomenti command line
 * 
 * @return 0 se successo, -1 se errore
 */
int parse_arguments(int argc, char *argv[], ProgramOptions *opts);

/**
 * Mostra usage e help
 */
void print_usage(const char *program_name);
void print_help(const char *program_name);

/**
 * Valida opzioni
 * 
 * @return true se valide, false se errori
 */
bool validate_options(const ProgramOptions *opts);

#endif // CLI_H
EOF

echo "✅ cli.h creato"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/08

cat > cli.c << 'EOF'
#include "cli.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

/**
 * Inizializza con default
 */
void init_options(ProgramOptions *opts) {
    opts->count = DEFAULT_COUNT;
    opts->subnet = NULL;
    opts->output = DEFAULT_OUTPUT;
    opts->only_public = true;   // Default: solo pubblici
    opts->show_classes = false;
    opts->verbose = false;
    opts->help = false;
}

/**
 * Mostra usage breve
 */
void print_usage(const char *program_name) {
    printf("Usage: %s [OPTIONS]\n", program_name);
    printf("Try '%s -h' for more information.\n", program_name);
}

/**
 * Mostra help completo
 */
void print_help(const char *program_name) {
    printf("RANDIP - Random IP Generator\n\n");
    printf("Usage: %s [OPTIONS]\n\n", program_name);
    printf("Options:\n");
    printf("  -n COUNT      Number of IP addresses to generate (default: %d)\n", 
           DEFAULT_COUNT);
    printf("  -s SUBNET     Subnet specification (CIDR, range, or single IP)\n");
    printf("                Examples: 192.168.1.0/24, 10.0.0.1-10.0.0.50, 8.8.8.8\n");
    printf("  -o FILE       Output file (default: %s)\n", DEFAULT_OUTPUT);
    printf("  -p            Generate only public IPs (default)\n");
    printf("  -c            Show IP classes in output\n");
    printf("  -v            Verbose output\n");
    printf("  -h            Show this help message\n");
    printf("\n");
    printf("Examples:\n");
    printf("  %s -n 100                     # 100 random public IPs\n", program_name);
    printf("  %s -n 50 -s 192.168.1.0/24    # 50 IPs in subnet\n", program_name);
    printf("  %s -n 20 -o output.txt -c     # With classes, custom output\n", program_name);
}

/**
 * Parse argomenti con getopt
 */
int parse_arguments(int argc, char *argv[], ProgramOptions *opts) {
    int opt;
    
    // Option string: 'n:' = -n richiede argomento, 'h' = -h no argomento
    const char *optstring = "n:s:o:pcvh";
    
    while ((opt = getopt(argc, argv, optstring)) != -1) {
        switch (opt) {
            case 'n':
                // Parse count
                opts->count = (size_t)atoi(optarg);
                if (opts->count == 0) {
                    fprintf(stderr, "Error: count must be > 0\n");
                    return -1;
                }
                break;
                
            case 's':
                // Subnet spec
                opts->subnet = optarg;
                break;
                
            case 'o':
                // Output file
                opts->output = optarg;
                break;
                
            case 'p':
                opts->only_public = true;
                break;
                
            case 'c':
                opts->show_classes = true;
                break;
                
            case 'v':
                opts->verbose = true;
                break;
                
            case 'h':
                opts->help = true;
                return 0;  // Help richiesto, non errore
                
            case '?':
                // Opzione sconosciuta o argomento mancante
                // getopt già stampa errore
                return -1;
                
            default:
                print_usage(argv[0]);
                return -1;
        }
    }
    
    // Controlla argomenti extra non-option
    if (optind < argc) {
        fprintf(stderr, "Error: unexpected argument '%s'\n", argv[optind]);
        return -1;
    }
    
    return 0;
}

/**
 * Valida opzioni parsed
 */
bool validate_options(const ProgramOptions *opts) {
    // Count valido
    if (opts->count == 0) {
        fprintf(stderr, "Error: count must be > 0\n");
        return false;
    }
    
    // Count ragionevole (max 10M per safety)
    if (opts->count > 10000000) {
        fprintf(stderr, "Error: count too large (max 10M)\n");
        return false;
    }
    
    // Output filename valido
    if (!opts->output || strlen(opts->output) == 0) {
        fprintf(stderr, "Error: output filename required\n");
        return false;
    }
    
    return true;
}
EOF

echo "✅ cli.c implementato"

## ✅ Test CLI

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/08

cat > test_cli.c << 'EOF'
#include <stdio.h>
#include <string.h>
#include "cli.h"

void print_options(const ProgramOptions *opts) {
    printf("  count: %zu\n", opts->count);
    printf("  subnet: %s\n", opts->subnet ? opts->subnet : "(none)");
    printf("  output: %s\n", opts->output);
    printf("  only_public: %s\n", opts->only_public ? "yes" : "no");
    printf("  show_classes: %s\n", opts->show_classes ? "yes" : "no");
    printf("  verbose: %s\n", opts->verbose ? "yes" : "no");
    printf("  help: %s\n", opts->help ? "yes" : "no");
}

void test_case(const char *desc, int argc, char *argv[]) {
    printf("\n=== %s ===\n", desc);
    printf("Command: ");
    for (int i = 0; i < argc; i++) {
        printf("%s ", argv[i]);
    }
    printf("\n\n");
    
    ProgramOptions opts;
    init_options(&opts);
    
    // Reset getopt (necessario per test multipli)
    optind = 1;
    
    int result = parse_arguments(argc, argv, &opts);
    
    if (result == 0) {
        if (opts.help) {
            printf("Help requested\n");
        } else {
            printf("Parse OK\n\n");
            print_options(&opts);
            
            if (validate_options(&opts)) {
                printf("\n✓ Valid options\n");
            } else {
                printf("\n✗ Invalid options\n");
            }
        }
    } else {
        printf("✗ Parse failed\n");
    }
}

int main() {
    // Test 1: Default
    {
        char *argv[] = {"randip"};
        test_case("TEST 1: Default (no args)", 1, argv);
    }
    
    // Test 2: Count only
    {
        char *argv[] = {"randip", "-n", "100"};
        test_case("TEST 2: Count", 3, argv);
    }
    
    // Test 3: Count + subnet
    {
        char *argv[] = {"randip", "-n", "50", "-s", "192.168.1.0/24"};
        test_case("TEST 3: Count + Subnet", 5, argv);
    }
    
    // Test 4: All options
    {
        char *argv[] = {
            "randip", "-n", "20", 
            "-s", "10.0.0.0/8",
            "-o", "output.txt",
            "-c", "-v"
        };
        test_case("TEST 4: All Options", 9, argv);
    }
    
    // Test 5: Help
    {
        char *argv[] = {"randip", "-h"};
        test_case("TEST 5: Help", 2, argv);
    }
    
    printf("\n\n=== HELP Output ===\n");
    print_help("randip");
    
    printf("\n✅ Test CLI completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_cli cli.c test_cli.c && ./test_cli

## 🔍 Analisi: optind e getopt State

### optind - Indice Globale

```c
extern int optind;  // Prossimo argv da processare
```

**Lifecycle:**
```
$ ./prog -n 100 -s 192.168.1.0/24 file.txt
           │     │                  │
optind:    1     3                  5

Dopo getopt loop:
  optind = 5  ← Punta a "file.txt" (non-option arg)
```

**Uso:**
```c
// Dopo getopt, processa argomenti rimanenti
for (int i = optind; i < argc; i++) {
    printf("Extra arg: %s\n", argv[i]);
}
```

### Reset per Test Multipli

```c
// getopt mantiene stato interno!
// Per test multipli, reset manuale:
optind = 1;  // Reset a primo argomento
```

## 🔍 Analisi: Exit Codes Standard

### POSIX Exit Codes

```c
0   - EXIT_SUCCESS - Esecuzione OK
1   - EXIT_FAILURE - Errore generico
2   - Usage error   - Argomenti invalidi
126 - Command found but not executable
127 - Command not found
128+N - Killed by signal N
```

### Uso in RANDIP

```c
int main(int argc, char *argv[]) {
    ProgramOptions opts;
    
    if (parse_arguments(argc, argv, &opts) != 0) {
        return 2;  // Usage error
    }
    
    if (opts.help) {
        print_help(argv[0]);
        return 0;  // Help non è errore
    }
    
    if (!validate_options(&opts)) {
        return 2;  // Validation error
    }
    
    // Genera IP...
    
    return 0;  // Success
}
```

## 📚 Recap

### ✅ CLI completo:

**Struct:**
```c
ProgramOptions {
    size_t count;         // -n COUNT
    const char *subnet;   // -s SUBNET
    const char *output;   // -o FILE
    bool only_public;     // -p
    bool show_classes;    // -c
    bool verbose;         // -v
    bool help;            // -h
}
```

**Funzioni:**
1. `init_options()` - Default values
2. `parse_arguments()` - getopt loop
3. `print_usage()` - Short help
4. `print_help()` - Full help
5. `validate_options()` - Sanity checks

### 🎓 Concetti:

- **argc/argv**: command line args
- **getopt**: POSIX option parser
- **optstring**: "n:s:oh" (: = arg required)
- **optarg**: current option argument
- **optind**: next argv index
- **Exit codes**: 0=ok, 2=usage error

### 🔑 Pattern:

```c
// 1. Init defaults
init_options(&opts);

// 2. Parse args
if (parse_arguments(argc, argv, &opts) != 0) {
    return 2;
}

// 3. Handle help
if (opts.help) {
    print_help(argv[0]);
    return 0;
}

// 4. Validate
if (!validate_options(&opts)) {
    return 2;
}

// 5. Execute
```

### ➡️ Ultimo Notebook!

**Notebook 9: Main - Integrazione Completa**
- Collega tutti i moduli
- Output file
- Statistiche finali
- Programma completo RANDIP

---

## 🎉 Notebook 8 Completato!

**Funzioni**: 5 CLI utilities  
**Concetti**: getopt, argc/argv, exit codes

Ultimo step → Notebook 9! 🏁